In [3]:
from helper import combine

_code = '''{{ command }} {{ head }}:
    {{ body }}'''


_for = combine({
    'main': _code,
    'command': 'for',
    'head': '{{ pointer }} in {{ collection }}',
})

_if = combine({
    'main': _code,
    'command': 'if',
    'head': '{{ condition }}',
})

_else = combine({
    'main': _code,
    'command': 'else',
    'head': '',
})


_def = combine({
    'main': _code,
    'command': 'def',
    'head': '{{ name }}({{ parameter }})'
})

_call = '{{ name }}({{ parameter }})'


invoke = combine({
    'main': _call,
    'parameter': ''
})

_var = '{{ name }} = {{ value }}\n'

_write = combine({
    'main': _call,
    'name':'print',
    'parameter':'{{ text }}'
})



def _import(imports):
    result = ''
    for i in imports:
        if(type(i) is list):
            parameters = {
                'main': 'from {{ module }} import {{ component }}\n',
                'module': i[0],
                'component': i[1]
            }
        else:
            parameters = {
                'main': 'import {{ module }}\n',
                'module': i
            }
        result += combine(parameters)
    return result
    
def parameter(input, space = False):
    result = ''
    for i in input.keys():
        result += combine({
            'main': '{{ name }} = {{ value }}{{ space }}',
            'name': i,
            'value': input[i],
            'space': ',\n' if space else ', '
        })
    if(len(result) > 0):
        result = result[:-2]
    return result


function = combine({
    'main'         : _def,
    'name'         : 'bar',
    'parameter'    : 'array',
    'body'         : _for,
    'collection'   : 'array',
    'pointer'      : 'i',
    'body 2'       : _for,
    'collection 2' : 'i',
    'pointer 2'    : 'j',
    'body 3'       : _write,
    'text'         : 'j'
})

print(function)



def bar(array):
    for i in array:
        for j in i:
            print(j)
